In [1]:
from transformers import GPT2ForSequenceClassification


model = GPT2ForSequenceClassification.from_pretrained("Dave12121/chat3Fsentiment", num_labels=3)
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")

In [2]:
from datasets import load_dataset

sentences_75 = load_dataset("financial_phrasebank", "sentences_66agree")


In [3]:
import torch
import tensorflow as tf
predLabels = []
refLabels = []
i = 1
for sample in sentences_75["train"]:
    refLabels.append(sample['label'])
    res = tokenizer(sample["sentence"], return_tensors="pt", padding=True, truncation=True, max_length=64)
    with torch.no_grad():
        result = model(**res).logits
    predicted_class_id = int(tf.math.argmax(result, axis=-1)[0])
    predLabels.append(predicted_class_id)
    i += 1
    if (i % 100) == 0:
        print(f"Progress: {i/len(sentences_75['train'])}")

2023-11-30 15:04:48.969004: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-30 15:04:49.169457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 15:04:49.169489: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 15:04:49.170057: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 15:04:49.278548: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

Progress: 0.023713540431586435
Progress: 0.04742708086317287
Progress: 0.0711406212947593
Progress: 0.09485416172634574
Progress: 0.11856770215793218
Progress: 0.1422812425895186
Progress: 0.16599478302110504
Progress: 0.18970832345269148
Progress: 0.21342186388427792
Progress: 0.23713540431586436
Progress: 0.2608489447474508
Progress: 0.2845624851790372
Progress: 0.30827602561062367
Progress: 0.3319895660422101
Progress: 0.35570310647379655
Progress: 0.37941664690538296
Progress: 0.4031301873369694
Progress: 0.42684372776855584
Progress: 0.4505572682001423
Progress: 0.4742708086317287
Progress: 0.4979843490633152
Progress: 0.5216978894949016
Progress: 0.5454114299264881
Progress: 0.5691249703580744
Progress: 0.5928385107896609
Progress: 0.6165520512212473
Progress: 0.6402655916528338
Progress: 0.6639791320844202
Progress: 0.6876926725160066
Progress: 0.7114062129475931
Progress: 0.7351197533791796
Progress: 0.7588332938107659
Progress: 0.7825468342423524
Progress: 0.8062603746739389
P

In [4]:
import evaluate
#1 neutral 2 positive

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

print(f1.compute(predictions = predLabels, references = refLabels, average = 'weighted'))
print(f1.compute(predictions = predLabels, references = refLabels, average = 'micro'))
print(f1.compute(predictions = predLabels, references = refLabels, average = 'macro'))
print(f1.compute(predictions = predLabels, references = refLabels, average = None))

{'f1': 0.8624989324750875}
{'f1': 0.8660184965615366}
{'f1': 0.8367449349130222}
{'f1': array([0.80773143, 0.90231171, 0.80019166])}
